In [ ]:
pip install pinecone-client

In [ ]:
!pip install -U pinecone-client sentence-transformers #https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [ ]:
#384차원 
from sentence_transformers import SentenceTransformer
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="")
index = pc.Index("yoo")
model = SentenceTransformer('all-MiniLM-L6-v2')

file_path = ".txt"
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.readlines()

# 각 텍스트 라인의 ID 생성
ids = [str(x) for x in range(len(data))]
embedded_datas = model.encode(data).tolist()
meta_datas =[{'text':text} for text in data]

records = zip(ids,embedded_datas, meta_datas)
index.upsert(vectors=records)

In [ ]:
def make_query(query):
    # Create the query vector using the model
    xq = model.encode(query).tolist()

    # Now query the Pinecone index using keyword arguments
    xc = index.query(vector=xq, top_k=5, include_metadata=True)

    # Extract results
    result = [{'score': match['score'], 'text': match['metadata']['text']} for match in xc['matches']]
    return result

# Example usage
response = make_query("월요일 3시 뭐하셔?")
print(response)